In [2]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_internal\cli\base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_internal\cli\req_command.py", line 245, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_internal\commands\install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-pack

## Python Dotenv

In [5]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv("req.env"), override=True)

#os.environ.get("OPENAI_API_KEY")

True

In [6]:
from langchain_openai import ChatOpenAI

llm =ChatOpenAI()
#output= llm.invoke("explain the OOP in one sentence")
#print(output.content)

In [8]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

messages= [
    SystemMessage(content="Sen pilota uçuş esnasında yardımcı olacak olan bir kopilot gibisin. Pilot sana ne sorarsa ona cevap verirsin."),
    HumanMessage(content="Uçağı nasıl yukarıya kaldıracağım?")
]

output= llm.invoke(messages)
print(output.content)

Uçağı yukarıya kaldırmak için ilk önce havalanma hızına ulaşmalısınız. Daha sonra kontrol yüzeylerini kullanarak uçağı yukarıya doğru kaldırabilirsiniz. Uçağı yukarıya kaldırmak için yükseklik dümesini aşağıya itmeniz gerekmektedir. Bu işlemi yaparken aynı zamanda hızı da kontrol etmeyi unutmayın.


## Caching LLM Responses

* We will go over caching in LangChain to boost performance and save costs
* Caching is the practice of storing frequently accessed data or results in a temprorary, faster storage layer. In the otherwords, caching is the practice of storing the responses of language models (such as GPT-4) for reuse. This technique is used to increase efficiency, reduce latency, and lower computational costs.
* Caching optimizes interactions with LLMs by reducing API calls and speeding up applications, resulting in a more efficient user experience.
*  Here's how it works:

Storing Responses: When a request is made to the language model and a response is generated, this response is stored in a cache along with the corresponding input or query.

Cache Lookup: For subsequent requests, the system first checks the cache to see if the response for the given query already exists.

Cache Hit: If the response is found in the cache (a cache hit), it is returned immediately without invoking the language model again.

Cache Miss: If the response is not found in the cache (a cache miss), the language model processes the query, generates a response, and stores this new response in the cache for future use.

* Benefits of LLM Caching:
1) Performance Improvement: Significantly reduces response time for repeated queries.
2) Cost Efficiency: Lowers the computational cost by reducing the number of times the language model needs to be invoked.
3) Resource Optimization: Frees up system resources by minimizing redundant computations.
* Implementation Considerations:
1) Cache Size: Determining the appropriate size of the cache to balance between storage costs and cache hit rate.
2) Eviction Policy: Choosing a policy for removing old or less frequently used entries (e.g., Least Recently Used (LRU), First In First Out (FIFO)).
3) Cache Invalidation: Deciding when to invalidate or refresh cache entries, especially if the underlying data or model changes.

LangChain provides two different caching In-memory cache and SQLite cache.

### 1) In-Memory Cache

In [23]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
from langchain.cache import InMemoryCache

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")#choose slower model

In [26]:
%%time

set_llm_cache(InMemoryCache())
prompt="give me a vegetables that is green."
llm.invoke(prompt)

CPU times: total: 15.6 ms
Wall time: 891 ms


'\n\nBroccoli'

In [28]:
%%time
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 998 μs


'\n\nBroccoli'

## 2) SQLite Cache

In [29]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

#First request(not in cache, takes Longer)
llm.invoke("Tell me a joke")

#Second request (cached, faster)
llm.invoke("Tell me a joke")

"\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything."

# Streaming the Response

* Streaming refers to the process of delivering the response in a continuous stream of data instead of sending the entire response at once.This allows the user to recieve the response piece by piece as it is generated, which can improve the user experience and reduce the overall latency

In [7]:
llm=ChatOpenAI()
prompt_For_Streaming="Give the song name that is the most popular from Metallica."
#print(llm.ivoke(prompt_For_Streaming).content)

for chunk in llm.stream(prompt_For_Streaming):
    print(chunk.content, end='', flush=True)

One of the most popular songs by Metallica is "Enter Sandman."

# Prompt Template

* A prompt refers to the input to the model
* Prompt templates are a way to create dynamic prompts for LLMs
* A prompt template takes a piece of text and injects a user's input into that piece of text
* In LangChain there are PromptTemplates and ChatPromptTemplates

In [35]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template ='''You are a computer engineer.
Write a few sentences about the following algorithm "{algo}" in {lang}.'''

prompt_template=PromptTemplate.from_template(template=template)

prompt=prompt_template.format(algo='A* Search', lang='Turkish')
prompt

'You are a computer engineer.\nWrite a few sentences about the following algorithm "A* Search" in Turkish.'

In [36]:
llm_prompt=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output=llm.invoke(prompt)
print(output.content)

A* Arama algoritması, bir graf içindeki en kısa yolun bulunmasında kullanılan bir arama algoritmasıdır. Bu algoritma, genişlik öncelikli arama ve en iyi ilk arama algoritmalarının bir kombinasyonunu kullanarak en etkili yolun bulunmasını sağlar. A* Arama algoritması, düşük maliyetli ve yüksek verimlilik sağlayarak birçok uygulamada tercih edilen bir algoritma türüdür.


# Chat Prompt Templates

* Chat Promt Templates are specifically designed for tasks that involve engaging in conversations.

* The chat completions API consists of a list of messages.

* Human Message prompt template is used to create dynamic prompts from the user's text.

* System message is used to format the system message.

In [10]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [11]:
chat_temlate=ChatPromptTemplate.from_messages(
    [
    SystemMessage(content="You respond only in the JSON format."),
    HumanMessagePromptTemplate.from_template("Top {n} countries in {area} by population.")
    ]
)

messages = chat_temlate.format_messages(n='10', area='Europe')
print(messages)

[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 10 countries in Europe by population.')]


In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output= llm.invoke(messages)
print(output.content)

{
    "countries": [
        {"name": "Germany", "population": 83783942},
        {"name": "France", "population": 65273511},
        {"name": "United Kingdom", "population": 67886011},
        {"name": "Italy", "population": 60461826},
        {"name": "Spain", "population": 46754778},
        {"name": "Ukraine", "population": 43733762},
        {"name": "Poland", "population": 37887768},
        {"name": "Romania", "population": 19237682},
        {"name": "Netherlands", "population": 17134872},
        {"name": "Belgium", "population": 11589623}
    ]
}


* As we can see, the respond is in JSON format according to the instructions that are defined.
* Some benefits of using prompt templates include reusability, scalability and accuracy.

# Simple Chain

* LLMs can be used for a variety of tasks, but they are often limited in what they can do on their own. For simple tasks or applications, you can use LLMs in isolation. For more complex tasks, it is often necessary to chain or combine different components.
* Chains in LangChain are like a series of steps or actions. Imagine it as a sequence of calls where each step can involve talking to a language model, using a tool or preparing data.
* Chains allow us to combine multiple components together to solve a specific task and build an entire LLM application.
* For example, imagine an answering questions app with retrieval and knowledge base. This app is made of chains, and these chains efficiently answer natural language questions by retrieving relevant documents from a knowledge base and leveraging an LLM to generate comprehensive and informative responses.
* LangChain provides a standard interface for chains, as well as some common implementations of chains to make it easy to create and use chains.

In [13]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [15]:
llm = ChatOpenAI()

template ='''You are a composer.
Write a few sentences about the following music "{music}" in {lang}.'''

prompt_template=PromptTemplate.from_template(template=template)

chain =LLMChain(
    llm=llm,
    prompt=prompt_template
)
output= chain.invoke({'music':'Dyers Eve', 'lang':'Turkish'})

In [16]:
print(output)

{'music': 'Dyers Eve', 'lang': 'Turkish', 'text': '"Dyers Eve", Metallica\'nın ünlü bir şarkısıdır. Şarkı, hızlı tempolu gitar riffleri ve etkileyici vokalleriyle bilinir. Şarkının sözleri, gençlik isyanını ve aile ilişkilerini ele alır ve dinleyicilere güçlü bir mesaj verir. "Dyers Eve", Metal müziğin enerjisini ve tutkusunu mükemmel bir şekilde yansıtan etkileyici bir parçadır.'}


In [18]:
template= 'What is the capital of {country}? List the top 3 places to visit in that city. Use bullet points'

prompt_template=PromptTemplate.from_template(template=template)

chain =LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country=input("Enter country: ")
output= chain.invoke(country)
print(output['text'])

Enter country:  turkey




> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of turkey? List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of Turkey is Ankara.

Top 3 places to visit in Ankara:
- Anitkabir (Mausoleum of Ataturk)
- Ankara Citadel
- Museum of Anatolian Civilizations


# Sequential Chains

* For more complex tasks, sequential chains are more suitable.
* With sequential chains, you can make a series of calls to one or more LLMs. You can take the output from one chain and use it as the input to another chain.
* There are two types of this.
1) SimpleSequentialChain represents a series of chains, where each individual chain has a single input and a single output, and the output of one step is used as input to the next. For example, you could create a simple sequential chain that first asks a question to an LLM, then takes the answer from the first LLM and uses it as input to a second LLM to get a morel detailed answer. 

In [2]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

In [6]:
llm1=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1=PromptTemplate.from_template(
    template = ' Linear regrasyonu anlat {regrasyon}.'
)

chain1=LLMChain(
    llm=llm1,
    prompt=prompt_template1
)

llm2=ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)
prompt_template2=PromptTemplate.from_template(
    template='örnekler vererek cevabı detaylandır {function}' 
)
chain2=LLMChain(
    llm=llm2,
    prompt=prompt_template2
)

#The second chain takes the output, whihc is a function from the first chain as input.

overall_chain=SimpleSequentialChain(chains=[chain1,chain2], verbose=True)

output=overall_chain.invoke('Linear Regrasyon')



> Entering new SimpleSequentialChain chain...
Linear regresyon, iki değişken arasındaki ilişkiyi modellemek için kullanılan istatistiksel bir tekniktir. Bu ilişki genellikle bağımlı değişken ile bir veya daha fazla bağımsız değişken arasındaki doğrusal ilişkiyi ifade eder.

Linear regresyon, en küçük kareler yöntemi kullanılarak gerçek verilerle uyumlu bir doğru veya düzlem oluşturur. Bu doğru veya düzlem, veriler arasındaki ilişkiyi en iyi şekilde temsil etmeye çalışır.

Linear regresyon genellikle tahmin yapmak veya değişkenler arasındaki ilişkiyi anlamak amacıyla kullanılır. Örneğin, bir bağımsız değişkenin diğer değişken üzerindeki etkisini anlamak veya bir değişkenin gelecekteki değerlerini tahmin etmek için kullanılabilir.

Linear regresyonun temel amacı, veri setindeki değişkenler arasındaki ilişkiyi modellemek ve bu ilişkiyi kullanarak gelecekteki değerleri tahmin etmektir. Bu nedenle, istatistiksel analizde sıkça kullanılan bir tekniktir.
Linear regresyon kullanılarak yapıla

# LangChain Agents

* LLMs are incredibly powerful, but they lack the ability to perform certain tasks that even the simplest application can do easily. For example, LLMs struggle with logic, calculation or communicating with other external components. For example, when we ask the calculation of (5.1 ** 7) to GPT4, it probably gives wrong answer. This is because GPT four is using an approximation while Python is using the exact mathematical formula. So, LLMs are powerful tool, but they are not perfect. The agent concept come out here. 

In [21]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from langchain_experimental.utilities import PythonREPL

# PythonREPL provides a way to execute Python code
python_repl=PythonREPL()
# Now, we can execute a python expression within the Repl environment

python_repl.run('print([n for n in range(1, 100) if n % 13==0])')
# there is no LLM here right now

'[13, 26, 39, 52, 65, 78, 91]\n'

In [25]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

In [28]:
llm=ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)

agent_executer= create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
# tool are essentially functions that agents can use to interact with the outside world. These can range from general utilities, such as a math calculator or a search function to other chains or even other agents.

agent_executer.invoke('calculate the square root of the factorial of 12 and display it with 4 decimal points')



> Entering new AgentExecutor chain...
To solve this, I will first calculate the factorial of 12 using the `math.factorial` function. Then, I will calculate the square root of that result using `math.sqrt`. Finally, I will format the result to display it with 4 decimal points using the `format` function. Let's start by importing the `math` module and calculating the factorial of 12.

Action: Python_REPL
Action Input: import math
print(math.factorial(12))
Observation: 479001600

Thought:Now that I have the factorial of 12, I will calculate the square root of this number using `math.sqrt`.

Action: Python_REPL
Action Input: print(math.sqrt(479001600))
Observation: 21886.105181141756

Thought:I have the square root of the factorial of 12. Now, I need to format this number to display it with 4 decimal points.

Action: Python_REPL
Action Input: print(format(21886.105181141756, '.4f'))
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [31]:
import math
math.sqrt(math.factorial(12))

21886.105181141756

LLM first makes a plan to solve the problem, then an observation and finally a thought

# LangChain Tools

* LangChain tools are like specialized apps for our LLM. They are tiny code modules that allow it to access information and services.
* These tools connect your LLM to search engines, databases, APIs, and more, expanding its knowledge and capabilities.
* We can build smarter, more versatile language models.

In [2]:
pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain.tools import DuckDuckGoSearchRun

search= DuckDuckGoSearchRun()

output =search.invoke("where was Freddie Mercury born")
print(output)

Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ... Freddie Mercury was born Farrokh Bulsara in Stone Town, Zanzibar (now Tanzania) on September 5, 1946. Learn more about his career, family, relationships, death and solo songs in this comprehensive article. Freddie Mercury was Britain's first Indian rock star. This fact and the nature of his sexuality were the two areas of his short life about which he would be intentionally obscure. Although he was born on the East African island of Zanzibar on September 5, 1946, his parents were both Indian; they were Parsees from Gujarat. Learn how Freddie Mercury, born Farrokh Bulsara in Stone Town, Zanzibar, beca

In [4]:
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
output= search.run("Metallica and James Hetfield")
print(output)

[snippet: Best of 2023: James Hetfield hasn't done many interviews for Metallica's new 72 Seasons album, and he doesn't really need to; ... This is the first electric guitar Hetfield used in Metallica; the Electra V that Hetfield bought in 1980 for $200 and was used on Kill 'Em All., title: James Hetfield on Metallica's new creative era: "I don't want to sit ..., link: https://www.musicradar.com/news/james-hetfield-on-metallica-s-new-creative-era-i-don-t-want-to-sit-there-and-create-the-songs-with-lars-anymore-i-want-everyone-to-be-a-part-of-it-and-be-in-it], [snippet: Metallica singer/guitarist James Hetfield has revealed that he's been writing new music during the heavy metal superstars' downtime. The 60-year-old, who co-founded Metallica with drummer Lars Ulrich in 1981, also revealed that he's been playing guitar every day since the end of the band's most recent tour in December 2023., title: Metallica's James Hetfield has been "writing lots of music ... - Louder, link: https://www

# ReAct-Reasoning and Acting

* ReAct is a new approach that combines reasoning(chain-of-thoughts prompting) and acting capabilities of LLMs
* With ReAct LLMs generate reasoning traces and task-specific actions in  an interleaved manner. This means that the model can reason about a task and then take actions in the real world to gather more information or complete the task.
* The combination of reasoning and acting allows react to overcome some of the limitations of existing LLMs, such as hallucinations and error propagation.
* LangChain agents are one of the key components that make long chain so useful. They take langchain tools, whihc we have looked at in some of the other videos and chains, and combine.
* LangChain hub is a hub for sharing and discovering resources related to LangChain. It stores and allows access to pre-trained prompts, chains and agents that can be used in LangChain project

In [17]:
pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
pip install -q wikipedia 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_openai import ChatOpenAI

In [20]:
# create LLM
llm= ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0)

#prepare the prompt template for asking question. This template will structure the way the questions are presented to the agent.
template='''
Answer the following questions as best you can.
Questions: {q}
'''
prompt_template=PromptTemplate.from_template(template)
prompt=hub.pull("hwchase17/react")#this is an object of type prompt template
#print(type(prompt))
#print(prompt.input_variables)
#print(prompt.template)

# 1. Python REPL Toll (for executing Python code)

python_repl=PythonAstREPLTool()
python_repl_tool=Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python Code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)

api_wrapper =WikipediaAPIWrapper()
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool=Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'

)

# 3. DuckDuckGo Search Tool (for general web searches)
search=DuckDuckGoSearchRun()
duckduckgo_tool=Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool cannot provide'
)
tools=[python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent= create_react_agent(llm, tools, prompt)

#struct agent executer to run react agent. The executor examines the input and determine the best tool to use based on the user's query.
agent_executor= AgentExecutor(
    agent=agent,
    tools = tools,
    verbose=True,
    handle_parsing_errors=True,# controls how the agent executor handles potential parsing error that might occur during the execution of tools, or the agent itself. 
    max_iterations=10 #using this argument prevents agents from running indefinitely, which could lead to performance issues, resource exhaustion, or getting stuck in loops.
    
)


In [21]:
question='Generate the first 20 numbers in the Fibonacci series'
output= agent_executor.invoke({
    'input':prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To generate the first 20 numbers in the Fibonacci series, I can write a simple Python program that iterates 20 times, each time calculating the next number in the series by summing up the two preceding numbers.

Action: Python REPL

Action Input:
```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        next_fib = fib_series[-1] + fib_series[-2]
        fib_series.append(next_fib)
    return fib_series[:n]

fibonacci(20)
```
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]I now know the first 20 numbers in the Fibonacci series.

Final Answer: The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.

> Finished chain.


In [22]:
print(output['input'])


Answer the following questions as best you can.
Questions: Generate the first 20 numbers in the Fibonacci series



In [23]:
output['output']

'The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.'

In [25]:
question2='who is the current prime minister of the UK?'
output= agent_executor.invoke({
    'input':prompt_template.format(q=question2)
})



> Entering new AgentExecutor chain...
I need to find the current Prime Minister of the United Kingdom as of my last update.

Action: Wikipedia
Action Input: Prime Minister of the United KingdomPage: Prime Minister of the United Kingdom
Summary: The prime minister of the United Kingdom is the head of government of the United Kingdom. The prime minister advises the sovereign on the exercise of much of the royal prerogative, chairs the Cabinet, and selects its ministers. As modern prime ministers hold office by virtue of their ability to command the confidence of the House of Commons, they sit as members of Parliament.
The office of prime minister is not established by any statute or constitutional document, but exists only by long-established convention, whereby the monarch appoints as prime minister the person most likely to command the confidence of the House of Commons. In practice, this is the leader of the political party that holds the largest number of seats in the Commons. The 

# Vector Embeddings

* Embeddings are the core of building LLM applications.
* Text embeddings are numeric representations of text and are used in NLP and ML tasks. Text embeddings can be used to measure the relatedness and similarity between two pieces of text. Relatedness is a measure of how closely two pieces of text are related in meaning.
* The distance between two embeddings or two vectors measures their relatedness which translates to the relatedness between the text concepts they represent.
* Similar embeddings or vector represent similar concepts, which translates to the relatedness between the text concepts they represent.
* There are two common approaches to measuring relatedness and similarity between text embeddings. These are Cosine similarity and Euclidean Distance.

# Vector Databases - Pinecone

* Vector databases are specialized database or data store specifically designed to manage such large quantities of data in numeric representation. Vector databases are a new type of database, designed to store and query unstructured data.
* There are several vector databases such as Pinecone, Milvus, Qdrant, ChromaDB. I am going to use Pinecone.
* Pinecone is a vector database designed for storing and querying high dimensional vectors. It provides fast and efficient semantic search over these vector embeddings.

In [27]:
pip install -q pinecone-client

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
pip install --upgrade -q pinecone-client

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
pip show pinecone-client

Name: pinecone-client
Version: 5.0.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: certifi, pinecone-plugin-inference, pinecone-plugin-interface, tqdm, typing-extensions, urllib3
Required-by: pinecone-plugin-inference
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv("Pinecone.env"), override=True)

True

In [4]:
from pinecone import Pinecone

pc= Pinecone()

# after creating the pinecone instance, we can check if it is successfully connected by attempting to perform an operation that requires authentication.
# For instance, we can create an index or list all existing indexes. If the operation completes without errors, your authentication is working.
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'churchill-2hcrrkb.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'churchill',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

# Pinecone Indexes

* An index is the highest level organizational unit of vector data in Pinecone
* It accepts and stores vectors, serves queires over the vectors it contains, and does other vector operations over its contents.
* Serverless Indexes: you do not configure or manage any compute or storage resources(they sclae automatically).
* Pod-based indexes: you choose one or more pre-configured units of hardware(pods).

In [5]:
from pinecone import ServerlessSpec

index_name='biai'
if index_name not in pc.list_indexes().names():
    pc.create_index(
    name=index_name,
    dimension=1536, # default dimension for text embedding, one of the recommended OpenAI's embedding models.
    metric='cosine', # algorithm to calculate distance between vectors.
    spec= ServerlessSpec(
        cloud='aws',
        region='us-east-1'
        )
    )
    print('Index is created successfully')

else:
    print(f'Index {index_name} already exists')


Index is created successfully


In [38]:
# delete index

index_name='biai'
if index_name in pc.list_indexes().names():
    print(f'Deleting index {index_name}')
    pc.delete_index(index_name)
    print('index is deleted')
else:
    print(f'index {index_name} does not exist')

Deleting index biai
index is deleted


In [6]:
index=pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [46]:
import random
vectors= [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids= list('abcde')

index_name='biai'
index=pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

* The upsert method returned the number of vectors inserted.

* When dealing with very long pieces of text, it is necessary to split the text into chunks. You also want to keep senamtically related pieces of text together.
* There are LangChain loaders that can be used to load data from almost any type of documents, CSV, Facebook chats, PDF, JSON, etc.

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import fitz

with fitz.open("den.pdf") as pdf:
    # Extract text from each page
    churchill = ""
    for page in pdf:
        churchill += page.get_text()
    
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [4]:
chunks= text_splitter.create_documents([churchill])

print(chunks[0])
print(chunks[3])

page_content='I am a fourth-year student at TED University. My department is Software Engineering.'
page_content='artiﬁcial intelligence. First internship was in Ministry of Justice. The department name'


In [5]:
print(f'now we have {len(chunks)} chunk')

now we have 29 chunk


### Embedding Cost

In [67]:
'''import tiktoken
def print_embedding_cost(texts):
    enc= tiktoken.encoding_for_model('text_embedding-ada-002')
    total_tokens=sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens/1000 *0.0004:.6f}')
    
print_embedding_cost(chunks)'''

"import tiktoken\ndef print_embedding_cost(texts):\n    enc= tiktoken.encoding_for_model('text_embedding-ada-002')\n    total_tokens=sum([len(enc.encode(page.page_content)) for page in texts])\n    print(f'Total tokens: {total_tokens}')\n    print(f'Embedding Cost in USD: {total_tokens/1000 *0.0004:.6f}')\n    \nprint_embedding_cost(chunks)"

In [7]:
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-small')

In [69]:
vector=embedding.embed_query('abc')
vector

[0.0026326572446609213,
 -0.011323257116496848,
 -0.009518612911691758,
 -0.0391218542455361,
 -0.03445101085960413,
 0.012172501283818792,
 -0.02125941545741194,
 -0.022816363252722596,
 0.01878245322511272,
 -0.00042484330187172293,
 0.0033845924076065883,
 0.019207074843112414,
 -0.00277773657687425,
 -0.004426686002148274,
 -0.01873999050451922,
 0.002970585771263353,
 0.027119201981446208,
 0.010700477998372584,
 0.010721709358669337,
 0.007537043032720129,
 -0.014444229827217079,
 0.01793320812646822,
 -0.006921341034694784,
 -0.015597787364824792,
 -0.02016955200069448,
 -0.0035579797545543077,
 0.009893695620321588,
 -0.020084628422152584,
 0.026298265363197376,
 -0.007338885998256838,
 0.007855510177542456,
 0.013828527829191733,
 -0.007381348253189064,
 -0.009957389701211843,
 -0.010431551159903955,
 -0.014182380108847367,
 -0.013644525500587553,
 -0.01590917599256437,
 0.01029000968924817,
 -0.0003295244755887975,
 0.02440161952842892,
 0.004522225726499822,
 0.0141045327190

In [8]:
vector=embedding.embed_query(chunks[0].page_content)
print(vector)

[-0.0407064954381451, -0.0659482831412245, -0.041176545103018054, 0.013020437743062288, 0.029166719913632393, 0.028015093019287406, 0.006013726631268852, 0.07036676787242312, -0.051423678461194874, -0.06359801991569901, -0.006057794020181328, -0.02919022276940506, -0.035065869657348224, -0.018285018762819055, -0.0014483474216766185, 0.014054551289866489, -0.02848514454680545, -0.02940174660871396, 0.008455058300639418, 0.03699308520425591, -0.00897211553970279, -0.010023856228336493, 0.045360008727070376, -0.004541876586650204, 0.03866176933766427, 0.009659566620472902, 0.022727008212435427, 0.018367276895378294, -0.02627590031856105, 0.018531796885786954, 0.004809218311435368, -0.02907271035318682, -0.0403539563268453, 0.02297378633540333, 0.03762765485953752, 0.01848479117424162, 0.02185741372471734, 0.024536709480479785, 0.01967167235224561, -0.028273620707496547, -0.018543546451028195, -0.018449536890582624, 0.011416385531706213, 0.02418417036917998, -0.042187156725372134, 0.017215

* This is the embedding of the first chunk

# Insert the Embeddings into a Pinecone Index

In [11]:
import pinecone
from langchain_community.vectorstores import Pinecone

pc =pinecone.Pinecone()

In [12]:
for i in pc.list_indexes().names():
    print('deleting all indexes...')
    pc.delete_index(i)
    print('Done')

deleting all indexes...
Done
deleting all indexes...
Done


In [13]:
index_name='deneme'
if index_name not in pc.list_indexes().names():
    print(f'creating index {index_name}...')
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=pinecone.PodSpec(
            environment='gcp-starter'
        )
    )
    print('Done')

creating index deneme...
Done


* Chunks is a list of text documents that have been obtained by calling the recursive character text splitter method.
* These smaller chunks will be indexed in pinecone to make it easier to search and retrieve relevant information.
* Embeddings is an instance of OpenAI embeddings class which is responsible for converting text data into embeddings using OpenAI's embedding model.
* These embeddings will be stored in the pinecone index and used for similarity search.
* index_name is used to identify the index in pinecone database and must already exist.

In [14]:
vector_store = Pinecone.from_documents(chunks, embedding, index_name=index_name)

# Asking Questions (Similarity Search)

* How it works?
* The user defines a query. The query is embedded into a vector.
* A similarity search is performed in the vector database, and the text behind the most similar vectors is the answer to the user's question.

In [82]:
query='who is my internship mate?'
result= vector_store.similarity_search(query)
print(result)

[Document(page_content='shall ﬁght on the landing grounds, we shall ﬁght in the ﬁelds and in the streets, we shall'), Document(page_content='ﬁght in the hills; we shall never surrender, and even if, which I do not for a moment'), Document(page_content='shall defend our Island, whatever the cost may be, we shall ﬁght on the beaches, we'), Document(page_content='ﬂag or fail. We shall go on to the end, we shall ﬁght in France, we shall ﬁght on the seas')]


In [83]:
for r in result:
    print(r.page_content)
    print('-' * 50)

shall ﬁght on the landing grounds, we shall ﬁght in the ﬁelds and in the streets, we shall
--------------------------------------------------
ﬁght in the hills; we shall never surrender, and even if, which I do not for a moment
--------------------------------------------------
shall defend our Island, whatever the cost may be, we shall ﬁght on the beaches, we
--------------------------------------------------
ﬂag or fail. We shall go on to the end, we shall ﬁght in France, we shall ﬁght on the seas
--------------------------------------------------


We need to answer in natural language. That's where LLM comes in, will retrieve the most relevant chunks of text and feed them to the language model for the final answer.

In [16]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm= ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})
# k equals 3 means that it will return the three most similar chunks to the user's query.

chain= RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [24]:
#query='where should we fight?'
query='where is my first intern and what is my internship topic? '
answer=chain.run(query)
print(answer)

Your first internship was completed in another company, which took 6 months to finish. Your current internship topic is cyber security in avionic systems.


In [25]:
engine= pyttsx3.init()
engine.say(answer)
engine.runAndWait()

In [20]:
import pyttsx3

In [21]:
engine= pyttsx3.init()

In [103]:
engine.say(answer)
engine.runAndWait()

In [ ]:
import speech_recognition as sr
import keyboard
import time

In [ ]:
recognizer = sr.Recognizer()
is_listening = False

def start_listening():
    global is_listening
    is_listening = True
    print("Listening... Press 'L' to stop.")
    
    while is_listening:
        try:
            with sr.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)

                text = recognizer.recognize_google(audio)
                text = text.lower()

                print(f'Recognized: {text}')
                
        except sr.UnknownValueError:
            # Bilinmeyen bir değer hatası, yani ses tanınamadı.
            print("Could not understand audio")
        except sr.RequestError:
            # Google API'ye erişim hatası.
            print("Could not request results; check your network connection")
            break

def stop_listening():
    global is_listening
    is_listening = False
    print("Stopped listening.")

# Ana döngü
print("Press 'L' to start/stop listening.")
while True:
    if keyboard.is_pressed('l'):
        if not is_listening:
            start_listening()
        else:
            stop_listening()
        # Tuşa basma işlemi tamamlandıktan sonra bir süre bekleyerek tekrar basılmasını engelleyin
        time.sleep(1)

# Text-to-Speech